In [1]:
# modules / libraries
import os
import sys
import warnings
from getpass import getpass

import matplotlib.pyplot as plt
import mysql.connector
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML, display

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [ ]:
# MySQL Ops : Python to MySQL Connection
passw = getpass('Enter password for MySQL: ')
try:
  db = mysql.connector.connect(host='localhost',user='root',password=passw,db='bike') # password='5678'
except:
  print('database not connected!')
else:
  print('database connected!',db)
  cursor = db.cursor()
  cursor.execute('show tables;')
  result = cursor.fetchall()
  print('#==== Tables =====#')
  for i in result:
    print(i[0])

# query to perform through mysql client
query = """select * from 2021_Jan limit 50;"""
cursor.execute(query)
result = cursor.fetchall()
print('#==== 2021_Jan =====#')
for i in result:
    print(i)

In [133]:
# list of files 
os.chdir('../dataset/') # datasets
files = sorted([i for i in os.listdir('../dataset/') if i.endswith('.csv')])

# Pandas preprocessing for the last 12 months of 2021 year
df1 = pd.read_csv(files[0])  # 2021_Jan
df2 = pd.read_csv(files[1])  # 2021_Feb
df3 = pd.read_csv(files[2])  # 2021_Mar
df4 = pd.read_csv(files[3])  # 2021_Apr
df5 = pd.read_csv(files[4])  # 2021_May
df6 = pd.read_csv(files[5])  # 2021_Jun
df7 = pd.read_csv(files[6])  # 2021_Jul
df8 = pd.read_csv(files[7])  # 2021_Aug
df9 = pd.read_csv(files[8])  # 2021_Sep
df10 = pd.read_csv(files[9])  # 2021_Oct
df11 = pd.read_csv(files[10])  # 2021_Nov
df12 = pd.read_csv(files[11])  # 2021_Dec

# one_year_data (2021 => Jan - Dec) # dataset to load approximately 33s query perfomed (max) # memory usage : 554.9+ MB
df = pd.DataFrame(np.vstack([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]),columns=df2.columns) # union_all (horizontally stacking)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5595063 entries, 0 to 5595062
Data columns (total 13 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   ride_id             object
 1   rideable_type       object
 2   started_at          object
 3   ended_at            object
 4   start_station_name  object
 5   start_station_id    object
 6   end_station_name    object
 7   end_station_id      object
 8   start_lat           object
 9   start_lng           object
 10  end_lat             object
 11  end_lng             object
 12  member_casual       object
dtypes: object(13)
memory usage: 554.9+ MB


In [ ]:
# first impression on the dataset (domain knowledge)
print('''
In a Dataset of 55,95,063 rows x 13 columns (55+ Lakhs or 5 Million Rows)

# Primary key constraint (df.ride_id.is_unique)
    ride_id					: tells about particular ride (primary key)

# Categorical (df.nunique())
    rideable_type			: electric_bike, classic_bike, docked_bike (3)
    member_casual			: member, casual (2)
    start_station_name		: biker starting point (847)
    end_station_name		: biker ending point (844)

# Numerical
    start_station_id		: each station has particular id defined (834)
    end_station_id			: each station has particular id defined (832)

# DateTime => no impurity in these fields
    started_at				: datetime field (46,77,998)
    ended_at				: datetime field (46,71,372)

# Geospatial 
    start_lat				: geographic-11 (39,4205)
    start_lng				: geographic-12 (37,4003)
    end_lat					: geographic-01 (44,5033)
    end_lng					: geographic-02 (40,5226)

# Missing values => there is a pattern in the data with the error of 0.3% (approx)
    start_station_name      : 69,0809
    start_station_id        : 69,0806
    end_station_name        : 73,9170
    end_station_id          : 73,9170
    end_lat                 : 4771
    end_lng                 : 477

    # if we drop the rows which having null in it (huge drop of rows)
        - total = 55,95,063
        - missing = 10,06,761 
        - percent = 17.993738408307465% 

    # most of rides are not considering end_station_name or end_station_id
    # end_lat + end_lng = end_station_name or end_station_id but there is a pattern of error in end_lng with 4294 = (end_lat)4771 - 477(end_lng)

# Findings / Patterns
    # start_station_name/end_station_name | start_station_id/end_station_id (category + numeric)
        - start_station_name (847) and start_station_id (834) has same values # missing_value_treatment
        - interchanging the start_station_name with start_station_id value or vice_versa by 13 rows vice-versa
        - end_station_name (844) and end_station_id (832) has same values # missing_value_treatment
        - interchanging the end_station_name with end_station_id value or vice_versa by 12 rows vice-versa

        - both start_station_name and start_station_id are missing can be replaced with highest frequency 
        - both end_station_name and end_station_id are missing can be replaced with highest frequency 

    # started_at/ended_at (datetime field)
        - started_at (46,77,998) out of 55,95,063 rows
            - so we assume there are 9,17,065 rows, means one or more rider has started_at same time 
            - we can say, most of the these 9,17,065 members are started with same place or different place using geo_location
        - ended_at (46,71,372) out of 55,95,063 rows
            - so we assume there are 9,23,691 rows, means one or more rider has ended_at same time 
            - we can say, most of the these 9,23,691 members are ended with same place or different place using geo_location

        - there is a deviation or else we can say, missed the ended_at or started_at with the error of 6626 ((started_at)46,77,998 - 46,71,371(ended_at))
        - there is a missing in the started_at point with the error of 9,17,065 (55,95,063 - 46,77,998(started_at))
        - there is a missing in the ended_at point with the error of 9,23,691 (55,95,063 - 46,71372(ended_at))

        - we can analyse the started_at and ended_at should be unique and how many members are able to come up with same_place
        - we can analyse, from which area how many are starting the ride and from which area how many are ending the ride
        - by the above analysis, we can ensure the docking station to increased or decreased
        - most of the rider are started at the same time but ended at different timing

    # start_lat/lng , end_lat/lng (rider starting or ending point) (geospatial)
        - we can say, start_lat/start_lng are dependent on [start_station_name,start_station_id]
        - we can say, end_lat/end_lng are dependent on [end_station_name,end_station_id]

        - start_lat(39,4205) out of 55,95,063 rows with the erorr(nulls) of 52,00,858 rows
        - start_lng(37,4003) out of 55,95,063 rows with the erorr(nulls) of 52,21,060 rows

        - end_lat(44,5033) out of 55,95,063 rows with the error(nulls) of 51,50,030 rows
        - end_lng(44,5033) out of 55,95,063 rows with the error(nulls) of 51,89,837 rows

        - start_lat(39,4205) - start_lng(37,4003) = 20,202 where as start_lat + start_lng = start_station_name or id
        - end_lat(39,4205) - end_lng(40,5226) = 11,021 where as end_lat + end_lng = end_station_name or id
        
        - start_lat/ start_lng are directly proportional to start_station_name or end_station_name | start_station_id or end_station_id
        - end_lat/ end_lng are directly proportional to end_station_name or end_station_name | end_station_id or end_station_id

        - we can say, start_lat + start_lng = start_station_name or start_station_id
        - we can say, end_lat + end_lng = end_station_name or end_station_id

    # missing_value treatment
        # inter-independent(5)
            - ride_id
            - rideable_type (independent or dependent based on all the features)
            - member_casual
            - started_at
            - ended_at
            
        # inter-dependent(8)
            - start_station_id  <=> start_station_name <=> start_lat + start_lng
            - end_station_id    <=> end_station_name   <=> end_lat + end_lng

        # model build [clustering (first), logistic_regression (next)]
            - x = (ride_id,rideabl_type,started_at,ended_at,start_station_id,ended_station_id,start_station_name, end_station_name, start_lat,start_lng,end_lat,end_lng)
            - y = member_casual (member=1,casual=0)

            # multi-collinearity
                - start_station_id <=> start_station_name <=> start_lat + start_lng <=> started_at
                - end_station_id   <=> end_station_name   <=> end_lat + end_lng <=> ended_at
    
    # observations
        - previously cyclistic bike-share program has 5,800 bicycles and has grown to a fleet of 5,824 with slight increment of 24 bicycles
        - previously cyclistic program has 600 docking stations and grown to a fleet of 693 with slight increment of 93 docking stations

        - locked in a network of starting and ending point of docking stations, but there is a probability of out of network
        - bikes can unlocked from one station and returned to another station, but there is a probability of out of station

        - one approach that helped make these things possible was the fexibility of pricing plans
            - single-ride passes |
            - full-day passes    | both are causal members
            - annual memberships

        - Casual : Customers who purchase single-ride or full-day passes are referred to as casual riders. 
        - Member : Customers who purchase annual memberships are Cyclistic members.

        - key objective : single-ride or full-day rider to acquire annual memberships

        - Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. (null-hypothesis proved)
        - Moreno notes that casual riders are already aware of the Cyclistic program and have chosen Cyclistic for their mobility needs. (null-hypothesis)
        
''')

In [ ]:
#==================================== Analysis ========================================#
# how many riders for each rideable_type?
rideable_count = pd.pivot_table(df,index='rideable_type',values='ride_id',aggfunc='count').reset_index()
rideable_casual_count = df.groupby('rideable_type')['member_casual'].value_counts().to_frame()

# how many member_casual are there?
casual_count = pd.pivot_table(df,index='member_casual',values='ride_id',aggfunc='count').reset_index()
casual_rideable_count = df.groupby('member_casual')['rideable_type'].value_counts().to_frame()

# quarter-wise results : q1(Jan,Feb,Mar) | q2(Apr,May,Jun) | q3(Jul,Aug,Sep) | q4(Oct,Nov,Dec) : 1m 20s query performed 
# date column['started_at','ended_at'] is clean and no TypeError such as date string doesn't meet the timestamp format
q1 = df[(pd.to_datetime(df['started_at']).apply(lambda x: x.month in (1,2,3))) | (pd.to_datetime(df['ended_at']).apply(lambda x: x.month in (1,2,3)))]
q2 = df[(pd.to_datetime(df['started_at']).apply(lambda x: x.month in (4,5,6))) | (pd.to_datetime(df['ended_at']).apply(lambda x: x.month in (4,5,6)))]
q3 = df[(pd.to_datetime(df['started_at']).apply(lambda x: x.month in (7,8,9))) | (pd.to_datetime(df['ended_at']).apply(lambda x: x.month in (7,8,9)))]
q4 = df[(pd.to_datetime(df['started_at']).apply(lambda x: x.month in (10,11,12))) | (pd.to_datetime(df['ended_at']).apply(lambda x: x.month in (10,11,12)))]

quarter = pd.DataFrame({'Q1': [q1.shape[0], q1['rideable_type'].mode()[0], q1['member_casual'].mode()[0]],
                        'Q2': [q2.shape[0], q2['rideable_type'].mode()[0], q2['member_casual'].mode()[0]],
                       'Q3': [q3.shape[0], q3['rideable_type'].mode()[0], q3['member_casual'].mode()[0]],
                        'Q4': [q4.shape[0], q4['rideable_type'].mode()[0], q4['member_casual'].mode()[0]]},
                       columns=['Q1', 'Q2', 'Q3', 'Q4'], index=['counts', 'rideable_type_mode', 'member_casual_mode'])

# Missing_Values impute with mean/median/mode (preprocessing)
missing = [(i, df.shape[0]-df[i].count()) for i in df.columns if df.shape[0] != df[i].count()]


#======================= Ask phase =======================#
- How do annual members and casual riders use Cyclistic bikes differently?
- Why would casual riders buy Cyclistic annual memberships?
- How can Cyclistic use digital media to influence casual riders to become members?

#========================= Task ==========================#
- What is the problem you are trying to solve?
- How can your insights drive business decisions?
- Identify the business task
- Consider key stakeholders

#===================== Deliverables =======================#
- A description of all data sources used
- Documentation of any cleaning or manipulation of data
- A summary of your analysis
- Supporting visualizations and key findings
- Your top three recommendations based on your analysis
- A clear statement of the business task (Deliverable)


In [ ]:
# How do annual members and casual riders use Cyclistic bikes differently
'''
#==================================== Code =========================================#
# how many riders for each rideable_type?
rideable_count = pd.pivot_table(df,index='rideable_type',values='ride_id',aggfunc='count').reset_index()
rideable_casual_count = df.groupby('rideable_type')['member_casual'].value_counts().to_frame()

# how many member_casual are there?
casual_count = pd.pivot_table(df,index='member_casual',values='ride_id',aggfunc='count').reset_index()
casual_rideable_count = df.groupby('member_casual')['rideable_type'].value_counts().to_frame()

# percentage-wise
round((df[(df['member_casual']=='member') & (df['rideable_type']=='docked_bike')].count()[0]/df[df['member_casual']=='member'].count()[0]) *100,2)

#==================================== Analysis ===========================================#
# rideable_type (approach 1)
	- classic_bike   : 32,51,028
		- member : 19,84,371
		- casual : 12,66,657
	- docked_bike    : 3,12,343
		- member : 1
		- casual : 3,12,342
	- electric_bike  : 20,31,692
		- member : 10,81,686
		- casual : 9,50,006

# member_casual (approach 2 : prefered)
	- casual : 25,29,005
		- classic_bike	: 12,66,657 (50%)
		- docked_bike	: 3,12,342 (12%)
		- electric_bike	: 9,50,006 (38%)
		
	- member : 30,66,058
		- classic_bike	: 19,84,371 (65%)
		- docked_bike	: 1 (0.0%)
		- electric_bike	: 10,81,686 (35%)

# Insights
	- there is low chance of being member since the biker is from docked_bike category
	- most of bikers love to ride through classic_bike category
	- there is low chance to ride through docked_bike category
	- 
'''

# Visualization
df.groupby(['member_casual', 'rideable_type'])['ride_id'].agg('count').plot(kind='bar')

# Why would casual riders buy Cyclistic annual memberships?
'''
Casual riders have single-day or full-day passes
Annual riders have annual passes

- benefits of annual subscription to single or full-day passes
- based on rider choice of commute or leisure
'''

# How can Cyclistic use digital media to influence casual riders to become members?
'''
Cyclistic encourge rider to use in any situation such as leisure, commute, road_trip, mountain_treking, etc.
influencial targeting based casual riders are important and achieved by tracking the data and helping on their day-to-day life

Design marketing strategy aimed at converting casual riders into annual members.

'''

# What is the problem you are trying to solve?
'''
These are the null values in the following featuers/columns
	start_station_name      : 69,0809
    start_station_id        : 69,0806
    end_station_name        : 73,9170
    end_station_id          : 73,9170
    end_lat                 : 4771
    end_lng                 : 477

- we have to fill the null values based on 
	- start_station_name <=> start_station_id <=> start_lat + start_lng
	- start_station_id <=> start_station_name <=> start_lat + start_lng
	- end_station_name <=> end_station_id <=> end_lat + end_lng
	- end_station_id <=> end_station_name <=> end_lat + end_lng
	- end_lat <=> end_lng <=> end_station_name <=> end_station_id
	- end_lng <=> end_lat <=> end_station_name <=> end_station_id

	# if there all are having null in there respective columns we have to replace with mode/median

# riding pattern (usage of rides)
# how much difference in distance are they covering interms of causal and member (bikes differently)

# we have to focus on these features
	- rideable_type
	- member_casual
	- started_at
	- ended_at

# the problem you are trying to solve is converting casual to annual member
'''

# How can your insights drive business decisions?
'''

'''
# Identify the business task
# Consider key stakeholders


In [138]:
# impute with their corresponding values in their respective features
'''
These are the null values in the following featuers/columns
	start_station_name      : 69,0809
    start_station_id        : 69,0806
    end_station_name        : 73,9170
    end_station_id          : 73,9170
    end_lat                 : 4771
    end_lng                 : 477

- we have to fill the null values based on 
	- start_station_name <=> start_station_id <=> start_lat + start_lng
	- start_station_id <=> start_station_name <=> start_lat + start_lng
	- end_station_name <=> end_station_id <=> end_lat + end_lng
	- end_station_id <=> end_station_name <=> end_lat + end_lng
	- end_lat <=> end_lng <=> end_station_name <=> end_station_id
	- end_lng <=> end_lat <=> end_station_name <=> end_station_id

	# if there all are having null in there respective columns we have to replace with mode/median
'''
# method 1: groupby
# lat,lng can be replaced with 2 decimals (42.168115666666665) = 15
cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
df[cols] = np.around(df[cols].astype(np.double), 1)

start_lat = df.groupby('start_lat')
start_lng = df.groupby('start_lng')

end_lat = df.groupby('end_lat')
end_lng = df.groupby('end_lng')


start_station_name = df.groupby('start_station_name') # not much significant
end_station_name = df.groupby('end_station_name') # not much significant

start_station_id = df.groupby('start_station_id')
end_station_id = df.groupby('end_station_id')



# method 2: template to find the pattern of missing values
# from itertools import combinations as cb
# l = map(set,cb(col, len(col)-1)) # represents unique feature for store the values of other featues
# z = map(lambda x: set(col)-x,l) # represents feature which we have to find out 
# lz = list(zip(map(lambda x: list(x)[0],z),map(list,l))) # template for identify the null values to replace with

l = ({'start_station_name', 'end_station_name', 'end_lat', 'end_station_id', 'start_station_id'}, {'start_station_name', 'end_station_name', 'end_lng', 'end_station_id', 'start_station_id'}, {'start_station_name', 'end_station_name', 'end_lng', 'end_lat', 'start_station_id'}, {
    'start_station_name', 'end_lng', 'end_lat', 'end_station_id', 'start_station_id'}, {'start_station_name', 'end_station_name', 'end_lng', 'end_lat', 'end_station_id'}, {'end_station_name', 'end_lng', 'end_lat', 'end_station_id', 'start_station_id'})
z = ({'end_lng'}, {'end_lat'}, {'end_station_id'}, {'end_station_name'}, {'start_station_id'}, {'start_station_name'})

d = {list(j)[0]: {x: np.nan for x in list(i)} for i, j in zip(l, z)}
for i in d.items():
	print(i)

# filling missing values
col = ['start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'end_lat', 'end_lng']

display(df[col].head(15))  # ['start_lat', 'start_lng', *col]


('end_lng', {'end_station_id': nan, 'start_station_id': nan, 'start_station_name': nan, 'end_lat': nan, 'end_station_name': nan})
('end_lat', {'end_lng': nan, 'end_station_id': nan, 'start_station_id': nan, 'start_station_name': nan, 'end_station_name': nan})
('end_station_id', {'end_lng': nan, 'start_station_id': nan, 'start_station_name': nan, 'end_lat': nan, 'end_station_name': nan})
('end_station_name', {'end_lng': nan, 'end_station_id': nan, 'start_station_id': nan, 'start_station_name': nan, 'end_lat': nan})
('start_station_id', {'end_lng': nan, 'end_station_id': nan, 'start_station_name': nan, 'end_lat': nan, 'end_station_name': nan})
('start_station_name', {'end_lng': nan, 'end_station_id': nan, 'start_station_id': nan, 'end_lat': nan, 'end_station_name': nan})


,start_lat,start_lng,end_lat,end_lng
0,41.9,-87.7,41.9,-87.7
1,41.9,-87.7,41.9,-87.7
2,41.9,-87.7,41.9,-87.7
3,41.9,-87.7,41.9,-87.7
4,41.9,-87.7,41.9,-87.7
5,41.9,-87.7,41.9,-87.7
6,41.9,-87.7,41.9,-87.7
7,41.9,-87.7,41.9,-87.7
8,41.9,-87.7,41.9,-87.7
9,41.9,-87.7,41.9,-87.7


In [168]:
col = ['start_lat', 'start_lng', 'start_station_id','start_station_name','end_lat','end_lng','end_station_id','end_station_name']
'''
first: find the id's base on lat and lng
where start_station_id can be filled using
    - start_lat + start_lng or start_station_name

where start_station_name can be filled using 
    - start_lat + start_lng or start_station_id 
'''


for i in df[col].itertuples():
    # data
    # print(i.start_station_name, i.end_station_name, i.end_lng, i.end_lat, i.end_station_id, i.start_station_id)

    # df['start_station_id'].isna().sum()
    if i.start_station_id is np.nan:
        leng = ' ' * abs(20-len(i)) if len(i) < 10 else ' '*20  # spacing

        # start_station_id using start_lat + start_lng
        print(f'start_lat {leng}: {i.start_lat}')
        print(f'start_lng {leng}: {i.start_lng}')
        

        print(f'start_station_id    {leng}: {i.start_station_id}')
        print(f'start_station_name  {leng}: {i.start_station_name}')

        print(f'end_lat {leng}: {i.end_lat}')
        print(f'end_lng {leng}: {i.end_lng}')
        print(f'end_station_id {leng}: {i.end_station_id}')
        print(f'end_station_name {leng}: {i.end_station_name}')

        # condition : 
        data = df.groupby(['start_lat', 'start_lng'], group_keys=False)[col].get_group((i.start_lat, i.start_lng))
        display(data)

        # if start_lat.groups.get(i.start_lat,None):
        #     print(start_lat[i.start_lat])
        
        break


start_lat            : 41.9
start_lng            : -87.7
start_station_id               : nan
start_station_name             : nan
end_lat            : 41.9
end_lng            : -87.7
end_station_id            : 13266
end_station_name            : Stave St & Armitage Ave


,start_lat,start_lng,start_station_id,start_station_name,end_lat,end_lng,end_station_id,end_station_name
0,41.9,-87.7,17660,California Ave & Cortez St,41.9,-87.7,NaN,NaN
1,41.9,-87.7,17660,California Ave & Cortez St,41.9,-87.7,NaN,NaN
2,41.9,-87.7,17660,California Ave & Cortez St,41.9,-87.7,NaN,NaN
3,41.9,-87.7,17660,California Ave & Cortez St,41.9,-87.7,NaN,NaN
4,41.9,-87.7,17660,California Ave & Cortez St,41.9,-87.7,NaN,NaN
...,...,...,...,...,...,...,...,...
5595029,41.9,-87.7,18058,Leavitt St & Chicago Ave,41.9,-87.7,NaN,NaN
5595042,41.9,-87.7,KA1504000114,Kedzie Ave & Chicago Ave,41.9,-87.7,NaN,NaN
5595043,41.9,-87.7,KA1504000114,Kedzie Ave & Chicago Ave,41.9,-87.7,NaN,NaN
5595044,41.9,-87.7,18058,Leavitt St & Chicago Ave,42.0,-87.7,NaN,NaN


In [ ]:
#==================== condition-base =====================#
# if i.start_station_name is np.nan:
#     print('start_station_name : ', i.start_station_name)
#     print('start_station_name : ',i.start_station_name)
#     print('end_station_id \t: ',i.end_station_id)
#     print('start_station_id : ',i.start_station_id)
#     print('end_lat \t : ',i.end_lat)
#     print('end_station_name : ',i.end_station_name)

#     # best significant feature is i.start_station_id
#     print(start_station_id['start_station_name'].groups.get(i.start_station_id,-1))
#     break

# if i.end_station_name is np.nan:
#     print(i.end_lng, i.end_station_id, i.start_station_id, i.start_station_name, i.end_lat)

# if i.start_station_id is np.nan:
#     print(i.end_lng, i.end_station_id, i.start_station_name, i.end_lat, i.end_station_name)

# if i.end_station_id is np.nan:
#     print(i.end_lng, i.start_station_id, i.start_station_name, i.end_lat, i.end_station_name)

# if i.end_lat is np.nan:
#     print(i.end_lng, i.end_station_id, i.start_station_id, i.start_station_name, i.end_station_name)

# if i.end_lng is np.nan:
#     print(i.end_station_id, i.start_station_id, i.start_station_name, i.end_lat, i.end_station_name)
pass


In [ ]:
# template format : hasmap (more than 15m time taking process)
dt = {}  #values : i.start_station_name, i.start_station_id, i.end_station_name,i.end_station_id,i.end_lat,i.end_lng

# retreive the data
for i in data.itertuples():
    # data
    # print(i.start_station_name, i.start_station_id, i.end_station_name,i.end_station_id,i.end_lat,i.end_lng,round(i.end_lat,2),round(i.end_lng,2))
    end_lat = np.round(i.end_lat,2) # i.end_lat
    end_lng = np.round(i.end_lng,2) # i.end_lng

    #=====================================================================================================================================#
    # ('start_station_name', {'end_station_name': nan, 'start_station_id': nan, 'end_lng': nan, 'end_lat': nan, 'end_station_id': nan})
    # if i.start_station_name in dt:
    #     convert into list
    #     if type(dt[i.start_station_name]['end_station_name']) != 'list':
    #         dt[i.start_station_name]['end_station_name'] = [dt[i.start_station_name]['end_station_name']]
    #     elif type(dt[i.start_station_name]['start_station_id']) != 'list':
    #         dt[i.start_station_name]['start_station_id'] = [dt[i.start_station_name]['start_station_id']]
    #     elif type(dt[i.start_station_name]['end_lng']) != 'list':
    #         dt[i.start_station_name]['end_lng'] = [dt[i.start_station_name]['end_lng']]
    #     elif type(dt[i.start_station_name]['end_lat']) != 'list':
    #         dt[i.start_station_name]['end_lat'] = [dt[i.start_station_name]['end_lat']]
    #     elif type(dt[i.start_station_name]['end_station_id']) != 'list':
    #         dt[i.start_station_name]['end_station_id'] = [dt[i.start_station_name]['end_station_id']]

    #     # adding to the list
    #     dt[i.start_station_name]['end_station_name'] = i.end_station_name
    #     dt[i.start_station_name]['start_station_id'] = i.start_station_id
    #     dt[i.start_station_name]['end_lng'] = i.end_lng
    #     dt[i.start_station_name]['end_lat'] = i.end_lat
    #     dt[i.start_station_name]['end_station_id'] = i.end_station_id
    # else:
    #     dt[i.start_station_name] = {'end_station_name': i.end_station_name, 'start_station_id': i.start_station_id, 'end_lng': i.end_lng, 'end_lat': i.end_lat, 'end_station_id': i.end_station_id}

    # ('end_station_name', {'start_station_name': nan, 'start_station_id': nan, 'end_lng': nan, 'end_lat': nan, 'end_station_id': nan})
    # if i.end_station_name in dt:
    #     # convert into list
    #     if type(df[i.end_station_name]['start_station_name']) != 'list':
    #         df[i.end_station_name]['start_station_name'] = [df[i.end_station_name]['start_station_name']]
    #     elif type(df[i.end_station_name]['start_station_id']) != 'list':
    #         df[i.end_station_name]['start_station_id'] = [df[i.end_station_name]['start_station_id']]
    #     elif type(df[i.end_station_name]['end_lng']) != 'list':
    #         df[i.end_station_name]['end_lng'] = [df[i.end_station_name]['end_lng']]
    #     elif type(df[i.end_station_name]['end_lat']) != 'list':
    #         df[i.end_station_name]['end_lat'] = [df[i.end_station_name]['end_lat']]
    #     elif type(df[i.end_station_name]['end_station_id']) != 'list':
    #         df[i.end_station_name]['end_station_id'] = [df[i.end_station_name]['end_station_id']]

    #     # adding to the list
    #     df[i.end_station_name]['start_station_name'] = i.start_station_name
    #     df[i.end_station_name]['start_station_id'] = i.start_station_id
    #     df[i.end_station_name]['end_lng'] = i.end_lng
    #     df[i.end_station_name]['end_lat'] = i.end_lat
    #     df[i.end_station_name]['end_station_id'] = i.end_station_id
    # else:
    #     df[i.end_station_name] = {'start_station_name': i.start_station_name, 'start_station_id': i.start_station_id, 'end_lng': i.end_lng, 'end_lat': i.end_lat, 'end_station_id': i.end_station_id}
    #=====================================================================================================================================#

    # ('start_station_id', {'start_station_name': nan, 'end_station_name': nan, 'end_lng': nan, 'end_lat': nan, 'end_station_id': nan})
    if i.start_station_id in dt:
        # convert into list
        # if type(df[i.start_station_id]['start_station_name']) != 'list':
        #     df[i.start_station_id]['start_station_name'] = [df[i.start_station_id]['start_station_name']]
        # elif type(df[i.start_station_id]['end_station_name']) != 'list':
        #     df[i.start_station_id]['end_station_name'] = [df[i.start_station_id]['end_station_name']]
        # elif type(df[i.start_station_id]['end_lng']) != 'list':
        #     df[i.start_station_id]['end_lng'] = [df[i.start_station_id]['end_lng']]
        # elif type(df[i.start_station_id]['end_lat']) != 'list':
        #     df[i.start_station_id]['end_lat'] = [df[i.start_station_id]['end_lat']]
        # elif type(df[i.start_station_id]['end_station_id']) != 'list':
        #     df[i.start_station_id]['end_station_id'] = [df[i.start_station_id]['end_station_id']]
            
        # adding to the list
        df[i.start_station_id]['start_station_name'] = i.start_station_name
        df[i.start_station_id]['end_station_name'] = i.end_station_name
        df[i.start_station_id]['end_lng'] = i.end_lng
        df[i.start_station_id]['end_lat'] = i.end_lat
        df[i.start_station_id]['end_station_id'] = i.end_station_id
    else:
        df[i.start_station_id] = {'start_station_name': i.start_station_name, 'end_station_name': i.end_station_name, 'end_lng': i.end_lng, 'end_lat': i.end_lat, 'end_station_id': i.end_station_id}

    # ('end_station_id', {'start_station_name': nan, 'end_station_name': nan, 'start_station_id': nan, 'end_lng': nan, 'end_lat': nan})
    if i.end_station_id in dt:
        # convert into list
        # if type(df[i.end_station_id]['start_station_name']) != 'list':
        #     df[i.end_station_id]['start_station_name'] = [df[i.end_station_id]['start_station_name']]
        # elif type(df[i.end_station_id]['end_station_name']) != 'list':
        #     df[i.end_station_id]['end_station_name'] = [df[i.end_station_id]['end_station_name']]
        # elif type(df[i.end_station_id]['start_station_id']) != 'list':
        #     df[i.end_station_id]['start_station_id'] = [df[i.end_station_id]['start_station_id']]
        # elif type(df[i.end_station_id]['end_lng']) != 'list':
        #     df[i.end_station_id]['end_lng'] = [df[i.end_station_id]['end_lng']]
        # elif type(df[i.end_station_id]['end_lat']) != 'list':
        #     df[i.end_station_id]['end_lat'] = [df[i.end_station_id]['end_lat']]

        # adding to the list
        df[i.end_station_id]['start_station_name'] = i.start_station_name
        df[i.end_station_id]['end_station_name'] = i.end_station_name
        df[i.end_station_id]['start_station_id'] = i.start_station_id
        df[i.end_station_id]['end_lng'] = i.end_lng
        df[i.end_station_id]['end_lat'] = i.end_lat
    else:
        df[i.end_station_id] = {'start_station_name': i.start_station_name,
                                'end_station_name': i.end_station_name, 'start_station_id': i.start_station_id, 'end_lng': i.end_lng, 'end_lat': i.end_lat}


    # ('end_lng', {'start_station_name': nan, 'end_station_name': nan, 'start_station_id': nan, 'end_lat': nan, 'end_station_id': nan})
    if end_lng in dt:
        # convert into list
        # if type(df[end_lng]['start_station_name']) != 'list':
        #     df[end_lng]['start_station_name'] = [df[end_lng]['start_station_name']]
        # elif type(df[end_lng]['end_station_name']) != 'list':
        #     df[end_lng]['end_station_name'] = [df[end_lng]['end_station_name']]
        # elif type(df[end_lng]['start_station_id']) != 'list':
        #     df[end_lng]['start_station_id'] = [df[end_lng]['start_station_id']]
        # elif type(df[end_lng]['end_lat']) != 'list':
        #     df[end_lng]['end_lat'] = [df[end_lng]['end_lat']]
        # elif type(df[end_lng]['end_station_id']) != 'list':
        #     df[end_lng]['end_station_id'] = [df[end_lng]['end_station_id']]

        # adding to the list
        df[end_lng]['start_station_name'] = i.start_station_name
        df[end_lng]['end_station_name'] = i.end_station_name
        df[end_lng]['start_station_id'] = i.start_station_id
        df[end_lng]['end_lat'] = end_lat
        df[end_lng]['end_station_id'] = i.end_station_id
    else:
        df[end_lng] = {'start_station_name': i.start_station_name, 'end_station_name': i.end_station_name,
                     'start_station_id': i.start_station_id, 'end_lat': end_lat, 'end_station_id': i.end_station_id}

    # ('end_lat', {'start_station_name': nan, 'end_station_name': nan, 'start_station_id': nan, 'end_lng': nan, 'end_station_id': nan})
    if end_lat in dt:
        # convert into list
        # if type(df[end_lat]['start_station_name']) != 'list':
        #     df[end_lat]['start_station_name'] = [df[end_lat]['start_station_name']]
        # elif type(df[end_lat]['end_station_name']) != 'list':
        #     df[end_lat]['end_station_name'] = [df[end_lat]['end_station_name']]
        # elif type(df[end_lat]['start_station_id']) != 'list':
        #     df[end_lat]['start_station_id'] = [df[end_lat]['start_station_id']]
        # elif type(df[end_lat]['end_lng']) != 'list':
        #     df[end_lat]['end_lng'] = [df[end_lat]['end_lng']]
        # elif type(df[end_lat]['end_station_id']) != 'list':
        #     df[end_lat]['end_station_id'] = [df[end_lat]['end_station_id']]

        # adding to the list
        df[end_lat]['start_station_name'] =  i.start_station_name
        df[end_lat]['end_station_name'] =  i.end_station_name,
        df[end_lat]['start_station_id'] =  i.start_station_id
        df[end_lat]['end_lng'] =  end_lng
        df[end_lat]['end_station_id'] =  i.end_station_id
    else:
        df[end_lat] = {'start_station_name': i.start_station_name, 'end_station_name': i.end_station_name,
                     'start_station_id': i.start_station_id, 'end_lng': end_lng, 'end_station_id': i.end_station_id}


print(dt)

In [ ]:
#======================= Prepare phase =======================#
# Where is your data located?
# How is the data organized?
# Are there issues with bias or credibility in this data? Does your data ROCCC?
# How are you addressing licensing, privacy, security, and accessibility?
# How did you verify the data’s integrity?
# How does it help you answer your question?
# Are there any problems with the data?

#================== Task =====================#
# Download data and store it appropriately.
# Identify how it’s organized.
# Sort and filter the data.
# Determine the credibility of the data.
# A description of all data sources used (Deliverable)


In [ ]:
#======================= Process phase =======================#
# What tools are you choosing and why?
# Have you ensured your data’s integrity?
# What steps have you taken to ensure that your data is clean?
# How can you verify that your data is clean and ready to analyze?
# Have you documented your cleaning process so you can review and share those results?

#================== Task =====================#
# Check the data for errors.
# Choose your tools.
# Transform the data so you can work with it effectively.
# Document the cleaning process.
# Documentation of any cleaning or manipulation of data (Deliverable)

#================== Hints =====================#
# 1. Open your spreadsheet and create a column called “ride_length.” 
# Calculate the length of each ride by subtracting the column “started_at” from the column “ended_at” (for example, =D2-C2) 
# and format as HH: MM: SS using Format > Cells > Time > 37: 30: 55.

# 2. Create a column called “day_of_week, ” and calculate the day of the week that each ride started using the “WEEKDAY” command 
# (for example, =WEEKDAY(C2, 1)) in each file. Format as General or as a number with no decimals, noting that 1 = Sunday and 7 = Saturday.


In [ ]:
#======================= Analyze phase =======================#
# How should you organize your data to perform analysis on it?
# Has your data been properly formatted?
# What surprises did you discover in the data?
# What trends or relationships did you find in the data?
# How will these insights help answer your business questions?

#================== Task =====================#
# Aggregate your data so it’s useful and accessible.
# Organize and format your data.
# Perform calculations.
# Identify trends and relationships.
# A summary of your analysis (Deliverable)

#================== Hints =====================#
# Where relevant, make columns consistent and combine them into a single worksheet.
# Clean and transform your data to prepare for analysis.
# Conduct descriptive analysis.
# Run a few calculations in one file to get a better sense of the data layout. Options:
    # Calculate the mean of ride_length
    # Calculate the max ride_length
    # Calculate the mode of day_of_week
# Create a pivot table to quickly calculate and visualize the data. Options:
    # Calculate the average ride_length for members and casual riders. Try rows = member_casual; Values = Average of ride_length.
    # Calculate the average ride_length for users by day_of_week. Try columns = day_of_week,Rows = member_casual, Values = Average of ride_length.
    # Calculate the number of rides for users by day_of_week by adding Count of trip_id to Values.
# Open another file and perform the same descriptive analysis steps. Explore different seasons to make some initial observations.
# Once you have spent some time working with the individual spreadsheets, merge them into a full-year view. Do this with the tool you have chosen to use to perform your final analysis, either a spreadsheet, a database and SQL, or R Studio.
# Export a summary file for further analysis

#================== Steps =====================#
# Method 1 : MySQL
# Import your data.
# Explore your data, perhaps looking at the total number of rows, distinct values, maximum, minimum, or mean values.
# Where relevant, use JOIN statements to combine your relevant data into one table.
# Create summary statistics.
# Investigate interesting trends and save that information to a table.

# Method 1 : Python
# Import your data.
# Make columns consistent and merge them into a single dataframe.
# Clean up and add data to prepare for analysis.
# Conduct descriptive analysis.
# Export a summary file for further analysis.


In [ ]:
#======================= Share phase =======================#
#======================= Act phase =======================#